In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime, date, timedelta
import os
import pymssql as mssql
import json
from multiprocessing import Pool    
from pathlib import Path
from functools import partial 
import logging, traceback
import sys
from tqdm import tqdm_notebook as tqdm
sys.path.insert(0, 'D:\庫存健診開發\code\ETFmain\inv_check_daily_prediction\core')


In [9]:
import ALL_STOCK_preprocess_function as preprocess

In [2]:
ETF = pd.read_csv('D:\庫存健診開發\data\ETF_basic.csv', converters={'ts': str, 'StockNo': str})
ETF['ts'] = pd.to_datetime(ETF['ts'])
index = pd.read_csv('D:\庫存健診開發\data\index_basic.csv', converters={'ts': str, 'StockNo': str})
index['ts'] = pd.to_datetime(index['ts'])

In [7]:
'0050' in ETF['StockNo'].unique()

True

In [10]:
df_list = [group[1] for group in ETF.groupby(ETF['StockNo'])]
timedf = pd.DataFrame(ETF['ts'].unique(), columns=['ts'])
timedf = timedf.sort_values(by='ts')
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for _, x in enumerate(tqdm(pool.imap_unordered(partial(preprocess.FillMissingTime, timedf=timedf, end_date=date(2019,9,23)), df_list), total=len(df_list)), 1):
                if x[0]:
                    output_list.append(x[1])
                else:
                    continue
                    
df = pd.concat(output_list)

In [11]:
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,VWAP,foreign_buy,investment_buy,dealer_buy,foreign_ratio,investment_ratio,dealer_ratio,corporation_ratio,start_date
0,2017-06-01,0050,元大台灣50,76.50,76.90,76.50,76.90,2221692,170334,5665.00,76.67,-101,-795,23,19.82,0,0,0,20040210
1,2017-06-02,0050,元大台灣50,77.30,77.30,77.05,77.20,3540934,273288,5665.00,77.18,-1782,0,1032,19.50,0,0,0,20040210
2,2017-06-03,0050,元大台灣50,77.25,77.40,77.25,77.40,2052295,158710,5665.00,77.33,200,0,646,19.54,0,0,0,20040210
3,2017-06-05,0050,元大台灣50,77.35,78.00,77.25,78.00,3281752,254540,5665.00,77.56,152,-243,748,19.56,0,0,0,20040210
4,2017-06-06,0050,元大台灣50,77.95,77.95,77.70,77.95,1203007,93684,5665.00,77.88,-4,0,-657,19.56,0,0,0,20040210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,2019-09-17,008201,BP上證50,107.40,107.40,107.40,107.40,100,11,35.49,107.40,0,0,0,0.40,0,0,0,20101208
568,2019-09-18,008201,BP上證50,106.55,107.30,106.55,107.30,300,32,35.49,106.92,0,0,1,0.40,0,0,0,20101208
569,2019-09-19,008201,BP上證50,107.30,107.30,106.70,106.70,200,21,35.49,107.00,0,0,1,0.40,0,0,0,20101208
570,2019-09-20,008201,BP上證50,106.70,107.90,106.70,107.90,600,64,35.49,106.90,0,0,6,0.40,0,0,0,20101208


In [12]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for x in pool.imap_unordered(partial(preprocess.merge_index, index=index), df_list):
                output_list.append(x)

df = pd.concat(output_list)

In [13]:
import VWAP_feature_function_rolling_week as FeatureEngineering

In [14]:
with open(r'D:\庫存健診開發\code\ETFmain\inv_check_daily_prediction\config\columns_dict.json', 'r') as fp:
    columns_dict = json.load(fp)

In [15]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for _, x in enumerate(tqdm(pool.imap_unordered(partial(FeatureEngineering.get_features, columns_dict=columns_dict, look_back=15, forward=5), df_list), total=len(df_list)), 1):
            output_list.append(x)

df = pd.concat(output_list, axis=0) 

In [16]:
df = df[df.ts.dt.date > date(2017,9,1)]
df = df[df['SOk15_5'] != 0]

In [17]:
df = df.fillna(0)

In [25]:
df['StockNo'] = df['StockNo'].apply(lambda x: x + '_ETF')

In [27]:
feature = FeatureEngineering.read_feature_list(r'D:\庫存健診開發\code\ETFmain\inv_check_daily_prediction\config\feature_dict.json', requirement='whole')

In [28]:
df[feature].to_csv(r'D:\OneDrive - SinoPac\training\ETF_whole.csv', index=False)

In [19]:
df.to_csv(r'D:\OneDrive - SinoPac\training\ETF_processed.csv', index=False)

In [5]:
feature = feature + ['Y']

In [6]:
train_df = pd.read_csv(r'D:\OneDrive - SinoPac\training\train_rolling_whole.csv', converters={'ts': str, 'StockNo': str})
train_df[feature].to_csv(r'D:\OneDrive - SinoPac\training\ETF_train.csv', index=False)

In [26]:
df

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,index_pvt,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5
68,2017-09-04,0053_ETF,元大電子,36.85,37.00,36.85,37.00,25000,924000,74.88,...,-7209.594316,0.0,0.0,-0.774526,-593.839075,0.684386,0.920000,0.909218,0.729931,0.714489
69,2017-09-05,0053_ETF,元大電子,37.29,37.31,36.98,37.11,14000,520000,74.88,...,25336.392926,0.0,0.0,-0.773877,-586.587992,0.761506,0.704918,0.807068,0.687686,0.705555
70,2017-09-06,0053_ETF,元大電子,37.10,37.10,36.95,36.95,5000,185000,74.88,...,35303.479272,0.0,0.0,-0.384653,-191.424500,0.741468,0.304348,0.555708,0.807453,0.739521
71,2017-09-07,0053_ETF,元大電子,36.95,37.25,36.95,36.95,6000,223000,74.88,...,115478.324609,0.0,0.0,-0.282074,31.574496,0.775261,0.543478,0.549593,0.818545,0.765862
72,2017-09-08,0053_ETF,元大電子,37.17,37.17,36.92,36.92,9000,333000,74.88,...,154755.016636,0.0,0.0,0.155633,469.466011,0.752771,0.391304,0.470449,0.825252,0.785659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,2019-09-09,008201_ETF,BP上證50,107.60,107.60,107.60,107.60,100,11000,35.49,...,156572.310917,0.0,0.0,3.083077,148.583993,0.673376,0.719512,0.741908,0.753623,0.708540
563,2019-09-10,008201_ETF,BP上證50,107.60,107.60,107.60,107.60,100,11000,35.49,...,184237.299805,0.0,0.0,3.319947,199.341939,0.673376,0.634921,0.688414,0.758720,0.725266
564,2019-09-11,008201_ETF,BP上證50,107.20,108.35,105.40,108.10,400,43000,35.49,...,167345.928601,0.0,0.0,2.128841,63.696781,0.478521,0.555224,0.621819,0.754857,0.735130
565,2019-09-12,008201_ETF,BP上證50,108.50,108.50,107.00,108.35,700,76000,35.49,...,231203.360686,0.0,0.0,2.788408,187.552291,0.490020,0.812903,0.717361,0.942792,0.804350


In [59]:
d[d.StockNo == '2330'][['ts', 'VWAP', 'close', 'close_return', 'VWAP_day5', 'VWAP_day5_return', 'VWAP_day5_return_5', 'Y']].iloc[:50]

,ts,VWAP,close,close_return,VWAP_day5,VWAP_day5_return,VWAP_day5_return_5,Y
767795,2007-07-20,69.19,69.4,-0.86,70.729704,-1.45,-1.85,0.0
767796,2007-07-23,68.56,68.7,-1.01,69.954579,-1.10,-3.26,0.0
767797,2007-07-24,70.16,71.3,3.78,69.805614,-0.21,-3.53,0.0
767798,2007-07-25,69.76,69.6,-2.38,69.549779,-0.37,-3.84,0.0
767799,2007-07-26,68.81,68.2,-2.01,69.310553,-0.34,-3.42,0.0
767800,2007-07-27,64.73,64.0,-6.16,67.004915,-3.33,-5.27,0.0
767801,2007-07-30,64.46,65.0,1.56,66.312738,-1.03,-5.21,0.0
767802,2007-07-31,64.89,65.0,0.00,65.560011,-1.14,-6.08,0.0
767803,2007-08-01,62.99,61.3,-5.69,64.688728,-1.33,-6.99,0.0
767804,2007-08-02,62.46,63.8,4.08,64.013841,-1.04,-7.64,0.0


In [52]:
d['ts'].max()

Timestamp('2019-09-16 00:00:00')

In [29]:
d = '20170915'
datetime.strptime(d, '%Y%m%d').date()

datetime.date(2017, 9, 15)